## bayesian global optimization with gaussian processes.

bayesian optimization is a global optimizer primarily used for black-box functions where we do not know how the output relates to the input parameters. it creates a prior distribution and evaluates the function at different values, updating this prior. these points are selected by balancing exploration and exploitation, as evaluations are costly. a gaussian process is used for the prior distribution.

an acquisition function acts as a heuristic to determine how desirable it is to evaluate a point. based on the acquisition function, we decide the next point to evaluate and update our model with the new exploration point.it is used in hyper paramter tunning.

In [34]:
!pip install bayesian-optimization

*exploration and exploitation*

---



In [35]:
from bayes_opt import BayesianOptimization

In [36]:
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

In [37]:
def black_box_function(x, y): # we generally wouldnt know details of this function
    return -x ** 2 - (y - 1) ** 2 + 1


In [38]:
# Bounds
pbounds = {'x': (2, 4), 'y': (-3, 3)}

In [39]:
optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds=pbounds,
    verbose=2,
    random_state=1,
)

In [40]:
optimizer

In [41]:
optimizer.maximize(  #we dont need to specify intt params
    init_points=2,  # selects 2 random points within the given bounds and initialise gp with this points
    n_iter=3,
)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
| 1         | -7.135    | 2.834     | 1.322     |
| 2         | -7.78     | 2.0       | -1.186    |
| 3         | -7.11     | 2.218     | -0.7867   |
| 4         | -12.4     | 3.66      | 0.9608    |
| 5         | -6.999    | 2.23      | -0.7392   |


In [42]:
optimizer.probe(
    params={"x": 0.5, "y": 0.7},
    lazy=True,
)


passing parameters as dict

In [43]:
optimizer.maximize(
    init_points=2,
    n_iter=3,
)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
| 6         | 0.66      | 0.5       | 0.7       |
| 7         | -16.13    | 2.294     | -2.446    |
| 8         | -8.341    | 2.373     | -0.9266   |
| 9         | -6.978    | 2.01      | 2.985     |


/usr/local/lib/python3.11/dist-packages/bayes_opt/target_space.py:421: UserWarning: 
Data point [0.5 0.7] is outside the bounds of the parameter space. 
  self.register(x, target)


| 10        | -3.06     | 2.013     | 0.9159    |
| 11        | -3.344    | 2.006     | 1.564     |


In [44]:
print(optimizer.space.keys)

['x', 'y']


passing parameters as iterable

In [45]:
optimizer.probe(
    params=[-0.3, 0.1],
    lazy=True,
)

In [46]:
optimizer.maximize(
    init_points=2,
    n_iter=3,
)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
| 12        | 0.1       | -0.3      | 0.1       |
| 13        | -7.392    | 2.794     | 0.2329    |
| 14        | -7.069    | 2.838     | 1.111     |


/usr/local/lib/python3.11/dist-packages/bayes_opt/target_space.py:421: UserWarning: 
Data point [-0.3  0.1] is outside the bounds of the parameter space. 
  self.register(x, target)


| 15        | -3.242    | 2.005     | 0.5277    |
| 16        | -3.009    | 2.001     | 1.076     |
| 17        | -3.02     | 2.0       | 1.14      |


In [47]:
optimizer.maximize(init_points=0, n_iter=0)


|   iter    |  target   |     x     |     y     |
-------------------------------------------------


In [48]:
logger = JSONLogger(path="./logs.log")
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [49]:
optimizer.maximize(
    init_points=2,
    n_iter=3,
)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
| 18        | -6.412    | 2.409     | 2.269     |
| 19        | -3.223    | 2.055     | 1.023     |
| 20        | -3.0      | 2.0       | 0.9936    |
| 21        | -3.0      | 2.0       | 0.9953    |
| 22        | -3.004    | 2.001     | 0.9707    |


In [50]:
from bayes_opt.util import load_logs

In [53]:
new_optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds={"x": (-2, 5), "y": (-2, 5)},
    verbose=2,
    random_state=7,
)
print(len(new_optimizer.space))

0


In [54]:
load_logs(new_optimizer, logs=["./logs.log"]);

In [60]:
new_optimizer.maximize(
    init_points=0,
    n_iter=10,
)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
| 31        | 0.9981    | 0.01837   | 0.9606    |
| 32        | 0.9999    | 0.0008573 | 0.9884    |
| 33        | 0.9987    | -0.03339  | 1.013     |
| 34        | 0.9931    | 0.03752   | 1.074     |
| 35        | 0.9984    | 0.02063   | 0.9654    |
| 36        | 0.9986    | -0.03667  | 1.01      |
| 37        | 0.9996    | 0.01924   | 1.005     |
| 38        | 0.9975    | 0.0344    | 1.037     |
| 39        | 0.9989    | 0.006706  | 0.968     |
| 40        | 0.9996    | 0.001861  | 1.02      |


In [56]:
new_optimizer.max

{'target': np.float64(0.8846070438181876),
 'params': {'x': np.float64(0.12923595239078223),
  'y': np.float64(0.6858487230784189)}}

In [57]:
for i, res in enumerate(new_optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))


Iteration 0: 
	{'target': np.float64(-6.412432296144895), 'params': {'x': np.float64(2.408904499463035), 'y': np.float64(2.268704618345673)}}
Iteration 1: 
	{'target': np.float64(-3.2226211374385345), 'params': {'x': np.float64(2.0547751863958523), 'y': np.float64(1.0228050610704136)}}
Iteration 2: 
	{'target': np.float64(-3.000041108993832), 'params': {'x': np.float64(2.0), 'y': np.float64(0.9935883704230357)}}
Iteration 3: 
	{'target': np.float64(-3.0000221766172164), 'params': {'x': np.float64(2.0), 'y': np.float64(0.995290794417714)}}
Iteration 4: 
	{'target': np.float64(-3.004163941710172), 'params': {'x': np.float64(2.000826474325388), 'y': np.float64(0.9707192665937128)}}
Iteration 5: 
	{'target': np.float64(-0.24601408189776208), 'params': {'x': np.float64(1.115941076406549), 'y': np.float64(0.9737398422252426)}}
Iteration 6: 
	{'target': np.float64(0.8846070438181876), 'params': {'x': np.float64(0.12923595239078223), 'y': np.float64(0.6858487230784189)}}
Iteration 7: 
	{'targe